# AI Assignment 3

In [1]:
# Boilerplate for AI Assignment — Knowledge Representation, Reasoning and Planning
# CSE 643

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import networkx as nx
from pyDatalog import pyDatalog
from collections import defaultdict, deque

## ****IMPORTANT****
## Don't import or use any other libraries other than defined above
## Otherwise your code file will be rejected in the automated testing

# ------------------ Global Variables ------------------
route_to_stops = defaultdict(list)  # Mapping of route IDs to lists of stops
trip_to_route = {}                   # Mapping of trip IDs to route IDs
stop_trip_count = defaultdict(int)    # Count of trips for each stop
fare_rules = {}                      # Mapping of route IDs to fare information
merged_fare_df = None                # To be initialized in create_kb()

# Load static data from GTFS (General Transit Feed Specification) files
df_stops = pd.read_csv('GTFS/stops.txt')
df_routes = pd.read_csv('GTFS/routes.txt')
df_stop_times = pd.read_csv('GTFS/stop_times.txt')
df_fare_attributes = pd.read_csv('GTFS/fare_attributes.txt')
df_trips = pd.read_csv('GTFS/trips.txt')
df_fare_rules = pd.read_csv('GTFS/fare_rules.txt')

# ---------------- DEBUG -----------------
# print(df_stops.columns.values.tolist())
# print(df_routes)
# print(df_stop_times)
# print(df_fare_attributes)
# print(df_trips)
# print(df_fare_rules)


## TESTING

In [2]:
test_inputs = {
    "direct_route": [
        ((2573, 1177), [10001, 1117, 1407]),  # Input -> Expected output
        ((2001, 2005), [10001, 1151])
    ],

    "forward_chaining": [
        ((22540, 2573, 4686, 1), [(10153, 4686, 1407)]),
        ((951, 340, 300, 1), [(1211, 300, 712), (10453, 300, 712), (387, 300, 712), (49, 300, 712), 
                              (1571, 300, 712), (37, 300, 712), (1038, 300, 712), (10433, 300, 712), 
                              (121, 300, 712)])
    ],
    "backward_chaining": [
        ((2573, 22540, 4686, 1), [(1407, 4686, 10153)]),
        ((340, 951, 300, 1), [(712, 300, 121), (712, 300, 1211), (712, 300, 37), (712, 300, 387),
                              (712, 300, 49), (712, 300, 10453), (712, 300, 1038), (712, 300, 10433),
                              (712, 300, 1571)])
    ],
    "pddl_planning": [
        ((22540, 2573, 4686, 1), [(10153, 4686, 1407)]),
        ((951, 340, 300, 1), [(1211, 300, 712), (10453, 300, 712), (387, 300, 712), (49, 300, 712), 
                        (1571, 300, 712), (37, 300, 712), (1038, 300, 712), (10433, 300, 712), 
                        (121, 300, 712)])
    ],
    "bfs_route": [
        ((22540, 2573, 10, 3), [(10153, 4686), (1407, 2573)]),
        ((4012, 4013, 10, 3), [(10004, 4013)])
    ],

    ### NOTE: The below values are just dummy values, the actual values are might differ! 
    "busiest_routes": [
        [(123, 456), (789, 234), (567, 235), (3456, 897), (345, 345)]
    ],
    "most_frequent_stops": [
        [(456, 456), (234, 765), (234, 765), (234, 657765), (3252, 35634)]
    ],
    "busiest_stops": [
        [(432243, 14543), (454235, 2452), (2452, 2454), (78568, 24352), (42352, 24532)]
    ],
    "stops_with_one_direct_route": [
        [((24527, 676), 542), ((243535, 8768), 2456), ((43262, 564), 65437),
         ((256, 56), 245), ((266, 256), 78)]
    ]
}

def check_output(expected, actual):
    """Function to compare expected and actual outputs."""
    return set(expected) == set(actual)

In [3]:
from pprint import pprint

# ------------------ Function Definitions ------------------

# Function to create knowledge base from the loaded data
def create_kb():
    """
    Create knowledge base by populating global variables with information from loaded datasets.
    It establishes the relationships between routes, trips, stops, and fare rules.
    
    Returns:
        None
    """
    global route_to_stops, trip_to_route, stop_trip_count, fare_rules, merged_fare_df

    # Create trip_id to route_id mapping
    for _, row in df_trips.iterrows():
        trip_id = row['trip_id']
        route_id = row['route_id']
        trip_to_route[trip_id] = route_id
    
    print("Trip to Route ID Mapping done")

    # Map route_id to a list of stops in order of their sequence
    for _, row in df_stop_times.iterrows():
        trip_id = row['trip_id']
        stop_id = row['stop_id']
        stop_sequence = row['stop_sequence']
        
        # Find the route_id from the trip_to_route mapping
        route_id = trip_to_route.get(trip_id)
        
        if route_id is not None:
            route_to_stops[route_id].append((stop_sequence, stop_id))
            stop_trip_count[stop_id] += 1

    print("Route ID to stop list done")

    # Sort stops by sequence within each route to maintain the correct order
    for route_id in route_to_stops:
        route_to_stops[route_id] = [stop_id for _, stop_id in sorted(route_to_stops[route_id])]

    print("Sorting of stop by sequence done")

    # Create fare rules for routes
    for _, row in df_fare_rules.iterrows():
        route_id = row['route_id']
        fare_id = row['fare_id']
        fare_rules[route_id] = fare_id

    print("Fare rules created")

    # Merge fare rules and attributes into a single DataFrame
    merged_fare_df = pd.merge(df_fare_rules, df_fare_attributes, on='fare_id', how='left')

    # ---------------- DEBUG -------------------
    # print(route_to_stops)
    # print(trip_to_route)
    # print(stop_trip_count)
    # print(fare_rules)
    # print(merged_fare_df)

    print("Knowledge base created successfully.")

create_kb()

Trip to Route ID Mapping done
Route ID to stop list done
Sorting of stop by sequence done
Fare rules created
Knowledge base created successfully.


In [4]:
# Function to find the top 5 busiest routes based on the number of trips
def get_busiest_routes():
    """
    Identify the top 5 busiest routes based on trip counts.

    Returns:
        list: A list of tuples, where each tuple contains:
              - route_id (str): The ID of the route.
              - trip_count (int): The number of trips for that route.
    """
    
    route_trip_count = defaultdict(int)
    
    # Count trips for each route using trip_to_route mapping
    for trip_id, route_id in trip_to_route.items():
        route_trip_count[route_id] += 1
    
    # Sort routes by trip count in descending order and return the top 5
    top_routes = sorted(route_trip_count.items(), key=lambda x: x[1], reverse=True)[:5]
    return top_routes

print(get_busiest_routes())

[(5721, 318), (5722, 318), (674, 313), (593, 311), (5254, 272)]


In [5]:
# Function to find the top 5 stops with the most frequent trips
def get_most_frequent_stops() -> list[tuple[int, int]]:
    """
    Identify the top 5 stops with the highest number of trips.

    Returns:
        list: A list of tuples, where each tuple contains:
              - stop_id (int): The ID of the stop.
              - trip_count (int): The number of trips for that stop.
    """

    _top5trips = sorted(stop_trip_count.items(), key=lambda x: x[1], reverse=True)[:5]
    return _top5trips

print(get_most_frequent_stops())

[(10225, 4115), (10221, 4049), (149, 3998), (488, 3996), (233, 3787)]


In [6]:
# Function to find the top 5 busiest stops based on the number of routes passing through them
def get_top_5_busiest_stops() -> list[tuple]:
    """
    Identify the top 5 stops with the highest number of different routes.

    Returns:
        list: A list of tuples, where each tuple contains:
              - stop_id (str): The ID of the stop.
              - route_count (int): The number of routes passing through that stop.
    """
    # Dictionary to map each stop to a set of routes passing through it
    stop_routes = defaultdict(set)
    
    # Populate stop_routes by iterating over route_to_stops
    for route_id, stops in route_to_stops.items():
        for stop_id in stops:
            stop_routes[stop_id].add(route_id)
    
    # Calculate the number of routes for each stop and sort in descending order
    _top5stops = sorted(
        ((stop_id, len(routes)) for stop_id, routes in stop_routes.items()),
        key=lambda x: x[1],
        reverse=True
    )[:5]
    
    return _top5stops

print(get_top_5_busiest_stops())

[(488, 102), (10225, 101), (149, 99), (233, 95), (10221, 86)]


In [7]:
# Function to identify the top 5 pairs of stops with only one direct route between them
def get_stops_with_one_direct_route():
    """
    Identify the top 5 pairs of consecutive stops (start and end) connected by exactly one direct route. 
    The pairs are sorted by the combined frequency of trips passing through both stops.

    Returns:
        list: A list of tuples, where each tuple contains:
              - pair (tuple): A tuple with two stop IDs (stop_1, stop_2).
              - route_id (str): The ID of the route connecting the two stops.
              - combined_trip_count (int): The combined frequency of trips through both stops.
    """
    # Dictionary to store each pair of stops and the route connecting them
    stop_pairs = defaultdict(list)
    
    # Step 1: Populate stop_pairs with consecutive stops for each route
    for route_id, stops in route_to_stops.items():
        for i in range(len(stops) - 1):
            # Create a tuple of consecutive stop pairs
            stop_pair = (stops[i], stops[i + 1])
            stop_pairs[stop_pair].append(route_id)

    # Step 2: Filter pairs that are connected by exactly one route
    one_route_pairs = {
        pair: routes[0] for pair, routes in stop_pairs.items() if len(routes) == 1
    }
    
    # Step 3: Calculate the combined trip count for each pair of stops
    # (assuming stop_trip_count provides the frequency of trips at each stop)
    combined_trip_counts = []
    for pair, route_id in one_route_pairs.items():
        stop_1, stop_2 = pair
        combined_count = stop_trip_count[stop_1] + stop_trip_count[stop_2]
        combined_trip_counts.append((pair, route_id, combined_count))

    # Step 4: Sort by combined trip count in descending order and select the top 5 pairs
    _top5pairs = sorted(combined_trip_counts, key=lambda x: x[2], reverse=True)[:5]

    return _top5pairs
print(get_stops_with_one_direct_route())

[((233, 148), 1433, 6440), ((10225, 11946), 5436, 6230), ((11044, 10120), 5916, 5732), ((11045, 10120), 5610, 5608), ((10225, 11160), 5814, 5492)]


In [8]:
# Brute-Force Approach for finding direct routes
def direct_route_brute_force(start_stop, end_stop):
    """
    Find all valid routes between two stops using a brute-force method.

    Args:
        start_stop (int): The ID of the starting stop.
        end_stop (int): The ID of the ending stop.

    Returns:
        list: A list of route IDs (int) that connect the two stops directly.
    """
    
    _totalRoutes = list()

    for _route, _stops in route_to_stops.items():
        if start_stop in _stops and end_stop in _stops:
            _totalRoutes.append(_route)
        
    return _totalRoutes

print(direct_route_brute_force(2001, 2005))

[10001, 1151]


In [9]:
# Initialize Datalog predicates for reasoning
pyDatalog.create_terms('RouteHasStop, DirectRoute, OptimalRoute, X, Y, Z, R, R1, R2')  
def initialize_datalog():
    """
    Initialize Datalog terms and predicates for reasoning about routes and stops.

    Returns:
        None
    """
    pyDatalog.clear()  # Clear previous terms
    print("Terms initialized: DirectRoute, RouteHasStop, OptimalRoute")  # Confirmation print

    # Define Datalog predicates
    DirectRoute(R, X, Y) <= RouteHasStop(R, X) & RouteHasStop(R, Y)  
    print("Rule for DirectRoute defined.")

    # create_kb()  # Populate the knowledge base
    add_route_data(route_to_stops)  # Add route data to Datalog
    
# Adding route data to Datalog
def add_route_data(route_to_stops):
    """
    Add the route data to Datalog for reasoning.

    Args:
        route_to_stops (dict): A dictionary mapping route IDs to lists of stops.

    Returns:
        None
    """

    print("Started adding facts")
    for route_id, stops in route_to_stops.items():
        for stop_id in stops:
            +RouteHasStop(route_id, stop_id)
    print("Finished adding facts")

initialize_datalog()

Terms initialized: DirectRoute, RouteHasStop, OptimalRoute
Rule for DirectRoute defined.
Started adding facts
Finished adding facts


In [10]:
# Function to query direct routes between two stops
def query_direct_routes(start, end):
    """
    Query for direct routes between two stops.

    Args:
        start (int): The ID of the starting stop.
        end (int): The ID of the ending stop.

    Returns:
        list: A sorted list of route IDs (str) connecting the two stops.
    """

    result = DirectRoute(R, start, end)
    return sorted(set([route_id[0] for route_id in result]))

print(query_direct_routes(2573, 1177))

[1117, 1407, 10001]


### Testing query_direct_routes

In [11]:
def test_query_direct_routes():
    for (start_stop, end_stop), expected_output in test_inputs["direct_route"]:
        actual_output = query_direct_routes(start_stop, end_stop)
        print(f"Test query_direct_routes ({start_stop}, {end_stop}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

print(test_query_direct_routes())

Test query_direct_routes (2573, 1177):  Pass
Test query_direct_routes (2001, 2005):  Pass
None


In [12]:
# Forward chaining for optimal route planning
def forward_chaining(start_stop_id: int,
                     end_stop_id: int,
                     stop_id_to_include: int,
                     max_transfers: int = 1) -> list[tuple[int, int]]:
    """
    Perform forward chaining to find optimal routes considering transfers.

    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID where a transfer occurs.
        max_transfers (int): The maximum number of transfers allowed.

    Returns:
        list: A list of unique paths (list of tuples) that satisfy the criteria, where each tuple contains:
              - route_id (int): The ID of the route.
              - stop_id (int): The ID of the stop.
    """ 

    OptimalRoute(X, Y, Z) <= (DirectRoute(X, start_stop_id, Z) & DirectRoute(Y, Z, end_stop_id))

    results = OptimalRoute(X, Y, stop_id_to_include)

    paths = list()
    paths.append((results[0][0], stop_id_to_include, results[0][1]))
    for res in range(1, len(results)):
        if results[res][0] == results[res - 1][1]:
            continue
        else:
            paths.append((results[res][0], stop_id_to_include, results[res][1]))

    return paths if paths else []
    

forward_chaining(951, 340, 300, 1)


[(121, 300, 712),
 (1038, 300, 712),
 (387, 300, 712),
 (10453, 300, 712),
 (49, 300, 712),
 (1211, 300, 712),
 (1571, 300, 712),
 (37, 300, 712),
 (10433, 300, 712)]

In [13]:
def test_forward_chaining():
    for (start_stop, end_stop, via_stop, max_transfers), expected_output in test_inputs["forward_chaining"]:
        actual_output = forward_chaining(start_stop, end_stop, via_stop, max_transfers)
        print(f"Test forward_chaining ({start_stop}, {end_stop}, {via_stop}, {max_transfers}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

test_forward_chaining()

Test forward_chaining (22540, 2573, 4686, 1):  Pass
Test forward_chaining (951, 340, 300, 1):  Pass


In [14]:
# Forward chaining for optimal route planning
def backward_chaining(start_stop_id: int,
                     end_stop_id: int,
                     stop_id_to_include: int,
                     max_transfers: int = 1) -> list[tuple[int, int]]:
    """
    Perform forward chaining to find optimal routes considering transfers.

    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID where a transfer occurs.
        max_transfers (int): The maximum number of transfers allowed.

    Returns:
        list: A list of unique paths (list of tuples) that satisfy the criteria, where each tuple contains:
              - route_id (int): The ID of the route.
              - stop_id (int): The ID of the stop.
    """ 

    OptimalRoute(X, Y, Z) <= (DirectRoute(X, start_stop_id, Z) & DirectRoute(Y, Z, end_stop_id))

    results = OptimalRoute(X, Y, stop_id_to_include)

    paths = list()
    paths.append((results[0][1], stop_id_to_include, results[0][0]))
    for res in range(1, len(results)):
        if results[res][0] == results[res - 1][1]:
            continue
        else:
            paths.append((results[res][1], stop_id_to_include, results[res][0]))

    return paths if paths else []
    

In [15]:
def test_backward_chaining():
    for (end_stop, start_stop, via_stop, max_transfers), expected_output in test_inputs["backward_chaining"]:
        actual_output = backward_chaining(start_stop, end_stop, via_stop, max_transfers)
        print(f"Test backward_chaining ({start_stop}, {end_stop}, {via_stop}, {max_transfers}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

test_backward_chaining()

Test backward_chaining (22540, 2573, 4686, 1):  Pass
Test backward_chaining (951, 340, 300, 1):  Pass


In [16]:
# PDDL-style planning for route finding
def pddl_planning(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
    """
    Implement PDDL-style planning to find routes with optional transfers.

    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID for a transfer.
        max_transfers (int): The maximum number of transfers allowed.

    Returns:
        list: A list of unique paths (list of tuples) that satisfy the criteria, where each tuple contains:
              - route_id (int): The ID of the route.
              - stop_id (int): The ID of the stop.
    """

    Action('board_route', R, X) <= RouteHasStop(R, X)
    Action('transfer_route', R1, R2, Z) <= DirectRoute(R1, start_stop_id, Z) & DirectRoute(R2, Z, end_stop_id)

    # Define the goal state
    result = Action('board_route', X, start_stop_id) & Action('transfer_route', X, Y, stop_id_to_include) & Action('board_route', Y, end_stop_id)
    
    paths = list()
    for res in result:
        paths.append((res[0], stop_id_to_include, res[1]))

    return paths if paths else []

pddl_planning(951, 340, 300, 1)

NameError: name 'Action' is not defined

In [ ]:
def test_pddl_planning():
    for (start_stop, end_stop, via_stop, max_transfers), expected_output in test_inputs["pddl_planning"]:
        actual_output = pddl_planning(start_stop, end_stop, via_stop, max_transfers)
        print(f"Test pddl_planning ({start_stop}, {end_stop}, {via_stop}, {max_transfers}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

test_pddl_planning()

In [ ]:
from pprint import pprint

# Function to filter fare data based on an initial fare limit
def prune_data(merged_fare_df, initial_fare):
    """
    Filter fare data based on an initial fare limit.

    Args:
        merged_fare_df (DataFrame): The merged fare DataFrame.
        initial_fare (float): The maximum fare allowed.

    Returns:
        DataFrame: A filtered DataFrame containing only routes within the fare limit.
    """

    return merged_fare_df[merged_fare_df['price'] <= initial_fare]



# Pre-computation of Route Summary
def compute_route_summary(pruned_df):
    """
    Generate a summary of routes based on fare information.

    Args:
        pruned_df (DataFrame): The filtered DataFrame containing fare information.

    Returns:
        dict: A summary of routes with the following structure:
              {
                  route_id (int): {
                      'min_price': float,          # The minimum fare for the route
                      'stops': set                # A set of stop IDs for that route
                  }
              }
    """
    pass  # Implementation here

# BFS for optimized route planning
def bfs_route_planner_optimized(start_stop_id: int,
                                end_stop_id: int,
                                initial_fare: int,
                                route_summary: int,
                                max_transfers: int = 3) -> list[tuple[int, int]]:
    """
    Use Breadth-First Search (BFS) to find the optimal route while considering fare constraints.

    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        initial_fare (float): The available fare for the trip.
        route_summary (dict): A summary of routes with fare and stop information.
        max_transfers (int): The maximum number of transfers allowed (default is 3).

    Returns:
        list: A list representing the optimal route with stops and routes taken, structured as:
              [
                  (route_id (int), stop_id (int)),  # Tuple for each stop taken in the route
                  ...
              ]
    """
    pass  # Implementation here

prune_data(merged_fare_df, 10)